In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ARmXIcOvzwuMAjyDhfgt")
project = rf.workspace("intellect-design-arena").project("classification-model-flash-n-f")
dataset = project.version(4).download("folder")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

train_data_dir = '/content/Classification-model-(Flash/N-F)-4/train'
validation_data_dir = '/content/Classification-model-(Flash/N-F)-4/valid'

batch_size = 32
image_size = (224, 224)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

model.save('flash_detection_vgg16.h5')


58889256/58889256 [==============================] - 0s 0us/step
Found 84 images belonging to 2 classes.
Found 1 images belonging to 2 classes.
Epoch 1/10
2/2 [==============================] - 24s 15s/step - loss: 4.9191 - accuracy: 0.4615
Epoch 2/10
2/2 [==============================] - 20s 12s/step - loss: 2.1226 - accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 25s 12s/step - loss: 1.3549 - accuracy: 0.4219
Epoch 4/10
2/2 [==============================] - 24s 12s/step - loss: 1.8162 - accuracy: 0.5469
Epoch 5/10
2/2 [==============================] - 20s 8s/step - loss: 1.6759 - accuracy: 0.6154
Epoch 6/10
2/2 [==============================] - 24s 12s/step - loss: 1.6243 - accuracy: 0.4375
Epoch 7/10
2/2 [==============================] - 26s 13s/step - loss: 1.3689 - accuracy: 0.6719
Epoch 8/10
2/2 [==============================] - 25s 13s/step - loss: 1.2509 - accuracy: 0.6094
Epoch 9/10
2/2 [==============================] - 24s 12s/step - loss: 0.8292 - a

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np


model = load_model('flash_detection_vgg16.h5')


img_path = '/content/Classification-model-(Flash/N-F)-4/valid/Images/Img12_jpeg.rf.3e4390b424126336f82021d0cff25fb6.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)

if predictions[0] > 0.5:
    print("Normal Image")
else:
    print("Flash Detected Image")


1/1 [==============================] - 0s 470ms/step
Normal Image
